In [1]:
import pandas as pd
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host="localhost",port=1521,sid='xe')
global conn
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
cursor = conn.cursor()                 #이게 없으면 오류가 나는데 이유가 뭔가요???

In [5]:
# 클래스 생성은 선택사항
class Member:
    '맴버 테이터 저장'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
        
    def as_dict(self):  #객체를 딕셔너리 테이터로 반환(csv파일 저장시)
        return {"name":self.name, "phone":self.phone, "email":self.email, "age":self.age, "grade":self.grade, "etc":self.etc}
    
    def to_list_style(self):  #객체를 "홍길동 010-8999-9999 e@e.com 20 3 까칠해" 
        temp = [self.name, self.phone, self.email, str(self.age), str(self.grade), self.etc]
        return ' '.join(temp)
    
    def __str__(self): #   *** 홍길동 010-8999-9999 e@e.com 20 3 까칠해
        return "{:3}\t{:15}\t{:15}\t{:3}\t{:}\t{}".format( self.name, self.phone,
                                                            self.email, self.age, self.grade, self.etc)

In [2]:
# 1. 입력
def insert_member():
    # 사용자로 부터 이름, 전화번호, 이메일, 나이, 등급(1-5), 기타특징 입력받아 DB에 추가한다.
    member_list=[]
    global conn
    curosr =conn.cursor()
    while True:
        try:
            name = input("이름 : (종료를 원하면 0)")
            if name == '0':
                break;
            phone = input("전화번호 :")
            email = input("이메일 :")
            age = int(input("나이 :"))
            if age<0:
                age=0
        except ValueError as e:
            print("유효하지 않는 나이는 입력시 0으로 초기화")
            age=0
        try:
            grade = int(input("등급(1-5) :"))
            if grade < 1:
                grade = 1
            if grade >5:
                grade =5
        except ValueError as e:
            print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
            grade = 1
        etc = input("기타정보 :")
        member = Member(name, phone, email, age, grade, etc)
        member_list.append(member)
    for member in member_list:    
        cursor.execute("INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)", member.as_dict())
    conn.commit()

In [3]:
#위의 메소드 test
insert_member()

NameError: name 'conn' is not defined

In [6]:
# 2. 전체 출력 pandas는 위에서 임포트 함
def select_all():
    global conn
    curosr =conn.cursor()
    #"SELECT * FROM MEMBER"의 결과를 출력(DataFrame형태 / 리스트형태)
    cursor.execute('SELECT * FROM MEMBER')
    data = cursor.fetchall()
    data_df = pd.DataFrame(data)
    data_df.columns =  [ row[0] for row in cursor.description]
    print(data_df)
    return data_df

In [42]:
# 위의 메소드 test
select_all()

,NAME,PHONE,EMAIL,AGE,GRADE,ETC
0,홍,1,2,3,4,5
1,길,1,2,3,4,5
2,동,1,2,3,4,5
3,이,1,6,3,4,
4,홍,길,동,1,2,3


In [7]:
# 3. 이름 검색
def select_name():
    #사용자로부터 검색하고자 하는 이름을 받아 결과 출력(DataFrame형태 / 리스트형태)
    global conn
    curosr =conn.cursor()
    name = input('검색할 이름은?')
    search_result = []
    cursor.execute('SELECT * FROM MEMBER')
    member_list = cursor.fetchall()
    for member in member_list:
        if member[0] == name:
            search_result.append(member)
    if len(search_result)==0:
        print('검색한 고객이 데이터에 존재하지 않습니다.')
    else:
        for member in search_result:
            print(member)

In [37]:
# 위의 메소드 test
select_name()

검색할 이름은?a
('a', 'a', 'a', 0, 1, 'a')


In [8]:
#4. 메일 삭제
def delete_email():
    # 사용자로부터 삭제할 메일을 입력받아 해당 메일이 있는지 확인하고
    # 해당 이메일이 없으면 없다 출력. 있으면 삭제하고 삭제했다 출력
    global conn
    curosr =conn.cursor()
    email = input('삭제할 이메일')
    cursor.execute('SELECT * FROM MEMBER where email = :email', {'email':email})
    member_list = cursor.fetchall()
    if(len(member_list))==0:
        print('메일 데이터가 존재하지 않아요')
    else:
        cursor.execute("DELETE FROM MEMBER WHERE EMAIL = :email", {'email':email})
        print(email,'을 삭제하였습니다')

In [41]:
delete_email()

삭제할 이메일hong@
hong@ 을 삭제하였습니다


In [9]:
# 5. CSV 내보내기
def save_csv():
    # 'SELECT * FROM MEMBER의 결과를 CSV로 내보내기(HEADER 포함)'
    import csv
    global conn
    curosr = conn.cursor()
    cursor.execute('SELECT * FROM MEMBER')
    member_list = cursor.fetchall()
    member_dict_list=[]
    
    for members in member_list:
        M_csv =Member(members[0],members[1],members[2],members[3],members[4],members[5])
        member_dict_list.append(M_csv.as_dict())
    
    fieldnames = ["name","phone","email","age","grade","etc"]
    filename = input('저장할 CSV 파일 이름은 ?')
    
    try :
        with open('data/'+filename+'.csv', 'w', newline='', encoding='cp949') as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
            dict_writer.writeheader()
            dict_writer.writerows(member_dict_list)
    except Exception as e:
        print('데이터 내보내기 예외 :', e)

In [52]:
#위의 메소드 test
save_csv()

저장할 CSV 파일 이름은 ?t


In [10]:
#0. 종료
def close_sql():
    global conn
    conn.close()
    print("종료합니다.")

In [11]:
def main():
    global conn
    while True:
        print("0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료", end=' ')
        fn = int(input("메뉴선택 : "))
        if fn == 1:
            insert_member()
        elif fn == 2:
            select_all()
        elif fn == 3:
            select_name()
        elif fn == 4:
            delete_email()
        elif fn == 5:
            save_csv()
        elif fn == 9:
            close_sql()
            break

In [15]:
if __name__=='__main__':
    import cx_Oracle
    import pandas as pd
    oracle_dsn = cx_Oracle.makedsn(host="localhost",port=1521,sid='xe')
    global conn
    conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
    cursor = conn.cursor()                 #이게 없으면 오류가 나는데 이유가 뭔가요???
    main()

0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료 메뉴선택 : 1
이름 : (종료를 원하면 0)홍길동
전화번호 :010-9999-9999
이메일 :HONG@HONG.com
나이 :23
등급(1-5) :3
기타정보 :까칠
이름 : (종료를 원하면 0)이순신
전화번호 :010-8888-8888
이메일 :yi@yi.com
나이 :25
등급(1-5) :4
기타정보 :까칠
이름 : (종료를 원하면 0)0
0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료 메뉴선택 : 2
  NAME          PHONE          EMAIL  AGE  GRADE ETC
0  홍길동  010-9999-9999  HONG@HONG.com   23      3  까칠
1  이순신  010-8888-8888      yi@yi.com   25      4  까칠
0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료 메뉴선택 : 3
검색할 이름은?홍길동
('홍길동', '010-9999-9999', 'HONG@HONG.com', 23, 3, '까칠')
0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료 메뉴선택 : 4
삭제할 이메일HONG@HONG.com
HONG@HONG.com 을 삭제하였습니다
0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료 메뉴선택 : 2
  NAME          PHONE      EMAIL  AGE  GRADE ETC
0  이순신  010-8888-8888  yi@yi.com   25      4  까칠
0:시작 | 1:입력 | 2:전체출력 | 3:이름찾기 | 4:삭제 | 5.내보내기(csv) | 9.종료 메뉴선택 : 5
저장할 CSV 파일 이름은 ?test_f_o
0:시작 | 1:입력 | 2:전체